# Multi-array: Utah 4

In [2]:
import sys
import os
import h5py 

# setup paths
iteration = 1 # which version of this tutorial to run (in case want results in different dirs)
NBname = 'color_cloud_initial{}'.format(iteration)

myhost = os.uname()[1] # get name of machine/
print("Running on Computer: [%s]" %myhost)

if myhost=='hoser':
    sys.path.insert(0, '/Users/dbutts/')
    dirname = os.path.join('.', 'checkpoints')
    datadir = '/Users/dbutts/V1/Conway/'
else:
    sys.path.insert(0, '/home/dbutts/Code/') 
    datadir = '/home/dbutts/ColorV1/Data/'  
    dirname = '/home/dbutts/ColorV1/CLRworkspace/' # Working directory 

import numpy as np
import scipy.io as sio
from copy import deepcopy

# plotting
import matplotlib.pyplot as plt

# Import torch
import torch
from torch import nn

# NDN tools
import NDNT.utils as utils # some other utilities
import NDNT.NDNT as NDN
from NDNT.modules.layers import *
from NDNT.networks import *
from time import time
import dill

from NTdatasets.generic import GenericDataset
import NTdatasets.conway.cloud_datasets as datasets
import NTdatasets.conway.multi_datasets as multidata

# Utilities
import ColorDataUtils.ConwayUtils as CU
import ColorDataUtils.EyeTrackingUtils as ETutils
from NDNT.utils import imagesc   # because I'm lazy
from NDNT.utils import ss        # because I'm real lazy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device0 = torch.device("cpu")
dtype = torch.float32

# Where saved models and checkpoints go -- this is to be automated
print( 'Save_dir =', dirname)
print(device)
%load_ext autoreload
%autoreload 2

Running on Computer: [ca1]
Invoking __init__.py for NDNT.utils
Save_dir = /home/dbutts/ColorV1/CLRworkspace/
cuda:0


## Data preprocessing

In [9]:
#expts = ['J220715','J220722','J220801','J220808']
expts = ['J220715']

pdir = datadir+'proc/'
NE = len(expts)
fns = []
for ee in range(NE):
    #fns.append("Jocamo_%s_full_CC_ETCC_nofix_v08"%edates[ee])
    fns.append( "%s/%s_CC_CC_v08"%(expts[ee], expts[ee]) )
    print(ee, expts[ee], ' | ', fns[ee] )

shifts = [None]*NE
metrics = [None]*NE
eLLsNULL = [None]*NE
eLLsGLM = [None]*NE
eLLsGQM = [None]*NE
eRFcenters = [None]*NE
edrifts = [None]*NE
tcs = [None]*NE
cell_lists = [None]*NE

0 J220715  |  J220715/J220715_CC_CC_v08


In [10]:
ee = 0
matdat = sio.loadmat(datadir+"%s/%s_UT_stimpos.mat"%(expts[ee], expts[ee]))
test = np.array(matdat['drift_terms'], dtype=np.float32)
matdat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'LLsNULL', 'LLsGLM', 'drift_terms', 'Dreg', 'RFcenters', 'top_corner', 'valUT', 'NX'])

In [11]:
for ee in range(NE):
    matdat = sio.loadmat(datadir+"%s/%s_UT_stimpos.mat"%(expts[ee], expts[ee]))
    cell_lists[ee] = np.array(matdat['val_lam'], dtype=np.int64).squeeze()
    cell_lists[ee] = np.array(matdat['valUT'], dtype=np.int64).squeeze()
    eLLsNULL[ee] = np.array(matdat['LLsNULL'], dtype=np.float32).squeeze()
    eLLsGLM[ee] = np.array(matdat['LLsGLM'], dtype=np.float32).squeeze()
    edrifts[ee] = np.array(matdat['drift_terms'], dtype=np.float32)
    eRFcenters[ee] = np.array(matdat['RFcenters'], dtype=np.int64)
    tcs[ee] = np.array(matdat['top_corner'], dtype=np.int64).squeeze()

    # Shifts
    matdat = sio.loadmat(datadir+"%s/%s_CC_CC_shifts_best.mat"%(expts[ee], expts[ee]))
    shifts[ee] = np.array(matdat['ETshifts'], dtype=np.int64)
    metrics[ee] = np.array(matdat['ETmetrics'], dtype=np.float32).squeeze()

In [12]:
num_lags=16

t0 = time()
mdata = multidata.MultiClouds(
    datadir=datadir, filenames=fns, eye_config=3, drift_interval=16,
    luminance_only=True, binocular=False, include_MUs=True, num_lags=num_lags,
    cell_lists=cell_lists,
    trial_sample=True)
t1 = time()
print("%0.2f sec elapsed"%(t1-t0))

for ee in range(NE):
    DFdat = sio.loadmat(datadir + expts[ee] + '/%s_CC_CC_DFupdate.mat'%expts[ee])
    mdata.updateDF( ee, DFdat['XDF'] )

  FILE_INFO: stim_locsLP list again -- ok but output check
167520 total time steps, 239 units
2.43 sec elapsed


In [ ]:
mdata.fileNT

In [13]:
L = 60
for ee in range(NE):
    mdata.build_stim( ee, top_corner=tcs[ee]-[15,15], L=L, shifts=shifts[ee] )
mdata.assemble_stim()

  Stim expansion for shift: [885, 477, 985, 577]
  Writing lam stim 0: overlap 60, 60
  Writing lam stim 1: overlap 60, 38
  Writing lam stim 2: overlap 18, 38
  Writing lam stim 3: overlap 18, 60
  Writing ETstim 0: overlap 22, 38
  Writing ETstim 1: overlap 22, 60
  Adding fixation point
  Shifting stim...


100%|█████████████████████████████████████████| 34/34 [00:19<00:00,  1.75it/s]


  CROP: New stim size: 60 x 60
  Done: expt 0
Stimulus assembly complete


In [ ]:
stas = torch.zeros([L**2, mdata.NC])
nspks = torch.zeros(mdata.NC)
bsize = 40
lag = 4
for bb in range(len(mdata.block_inds)//bsize):
    ts =  np.arange(bb*bsize, (bb+1)*bsize)
    Reff = torch.mul(mdata[ts]['robs'], mdata[ts]['dfs'])
    nspks += torch.sum(Reff, axis=0)
    stas += (mdata[ts]['stim'][:-lag, ...].T @ Reff[lag:,:])

stas0 = (stas/nspks).reshape([L,L,mdata.NC]).detach().numpy()
stas0.shape

In [ ]:
ss(117,5)
for cc in range(mdata.NC):
    #for clr in range(3):
    plt.subplot(117,5, cc+1)
    imagesc(stas0[:,:,cc])
    plt.title('c%d'%cc)
plt.show()

In [ ]:
## Difference in cells included in 7/15 dataset
set(test)-set(cell_lists[0])

In [ ]:
eRFcenters[0].shape